In [2]:
import csv 
from collections import defaultdict 
from multiprocessing import Pool, cpu_count  
#Mapper  
def mapper(rows): 
    mapped = [] 
    for row in rows: 
        country = row["country_txt"] 
        year = row["iyear"] 
        if country and year: 
            key = (country, year) 
            mapped.append((key, 1)) 
    return mapped 
#Shuffle  
def shuffle(mapped_lists): 
    grouped = defaultdict(list) 
    for mapper_output in mapped_lists: 
        for key, value in mapper_output: 
            grouped[key].append(value) 
    return grouped 
#Reducer 
def reducer(item): 
    key, values = item 
    total = sum(values) 
    return (key[0], key[1], total) 
#Main 
def chunkify(data, n): 
    avg = len(data) // n 
    return [data[i:i+avg] for i in range(0, len(data), avg)] 
 
def run_mapreduce(input_csv): 
    with open(input_csv, encoding="latin1") as f: 
        reader = csv.DictReader(f) 
        rows = list(reader) 
    n = max(1, cpu_count() // 2) 
    chunks = chunkify(rows, n) 
    with Pool(n) as p: 
        mapped = p.map(mapper, chunks) 
    grouped = shuffle(mapped) 
    items = list(grouped.items()) 
    with Pool(n) as p: 
        reduced = p.map(reducer, items) 
    reduced.sort(key=lambda x: (x[0], x[1])) 
    with open("country_year_attacks.csv", "w", newline='') as f: 
        writer = csv.writer(f) 
        writer.writerow(["Country", "Year", "Total_Attacks"]) 
        for row in reduced: 
            writer.writerow(row) 
    # Top-10 by attack frequency 
    reduced_sorted = sorted(reduced, key=lambda x: x[2], reverse=True) 
    top10 = reduced_sorted[:10] 
    with open("top10_countries.csv", "w", newline='') as f: 
        writer = csv.writer(f) 
        writer.writerow(["Country", "Year", "Total_Attacks"]) 
        for row in top10: 
            writer.writerow(row) 
    return "MapReduce Completed Successfully!" 
if __name__ == "__main__": 
    print(run_mapreduce("gtd.csv"))

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'gtd.csv'